## Import


In [2]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport

df = pd.read_csv("insurance.csv")
profile = ProfileReport(df, title="My Data Report")

# Save as HTML file to view in browser
profile.to_file("data_report2.html")
print("Report saved as data_report2.html - open it in your browser")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 67.29it/s]

Report saved as data_report2.html - open it in your browser


In [7]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()




np.int64(0)

In [20]:
# Check column names and shape
print("Columns in dataframe:")
print(df.columns.tolist())
print(f"\nShape: {df.shape}")
print(f"\nFirst few rows:")
df.head()
df.shape

Columns in dataframe:
['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']

Shape: (1337, 7)

First few rows:


(1337, 7)

## encoding 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
X=df.drop('charges',axis=1)
Y=df['charges']
encoder=OneHotEncoder(sparse_output=False)
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
X_categorical = encoder.fit_transform(X[categorical_columns])
X_preprocessed=np.hstack((X[numerical_columns], X_categorical))
X_preprocessed=pd.DataFrame(X_preprocessed,columns=numerical_columns.tolist() + encoder.get_feature_names_out(categorical_columns).tolist())
y=pd.DataFrame(Y, columns=['charges'])
print(X_preprocessed.head())
print(y.head())
##label encoder

    age     bmi  children  sex_female  sex_male  smoker_no  smoker_yes  \
0  19.0  27.900       0.0         1.0       0.0        0.0         1.0   
1  18.0  33.770       1.0         0.0       1.0        1.0         0.0   
2  28.0  33.000       3.0         0.0       1.0        1.0         0.0   
3  33.0  22.705       0.0         0.0       1.0        1.0         0.0   
4  32.0  28.880       0.0         0.0       1.0        1.0         0.0   

   region_northeast  region_northwest  region_southeast  region_southwest  
0               0.0               0.0               0.0               1.0  
1               0.0               0.0               1.0               0.0  
2               0.0               0.0               1.0               0.0  
3               0.0               1.0               0.0               0.0  
4               0.0               1.0               0.0               0.0  
       charges
0  16884.92400
1   1725.55230
2   4449.46200
3  21984.47061
4   3866.85520


In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_preprocessed,y,test_size=0.2,random_state=42)

In [19]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

# Define the model
reg = LinearRegression()

# Define the parameter grid
param_grid = {
    'fit_intercept': [True, False]   # Whether to calculate the intercept
}

# Apply GridSearchCV
grid_search = GridSearchCV(
    estimator=reg,
    param_grid=param_grid,
    cv=5,                # 5-fold cross-validation
    scoring='r2',        # For regression tasks, use R² or neg_mean_squared_error
    n_jobs=-1
)

# Fit the model
grid_search.fit(x_train, y_train)

# Best parameters
print("Best Parameters:", grid_search.best_params_)

# Best estimator
best_reg = grid_search.best_estimator_

# Predictions with best model
lr_pred = best_reg.predict(x_test)

# Evaluation metrics
print("R2 Score:", r2_score(y_test, lr_pred))
print("Mean absolute Error:", mean_absolute_error(y_test, lr_pred))
print("mean squared error", mean_squared_error(y_test, lr_pred))
print("Mean Squared Error root :", np.sqrt(mean_squared_error(y_test, lr_pred)))



Best Parameters: {'fit_intercept': True}
R2 Score: 0.8069287081198011
Mean absolute Error: 4177.045561036322
mean squared error 35478020.67523561
Mean Squared Error root : 5956.342894363588


Linear Regression Model – Full Observation
Model Setup
Algorithm Used: Linear Regression

Best Parameters (via GridSearchCV, cv=5):

fit_intercept = True → The model includes an intercept term, which improves its ability to capture baseline insurance charges.

Performance Metrics
R² Score: 0.807

This means the model explains about 80.7% of the variance in insurance charges. It indicates a strong fit, though not perfect.

Mean Absolute Error (MAE): 4177.05

On average, the model’s predictions deviate from the actual charges by about $4,177.

Mean Squared Error (MSE): 35,478,020.68

The squared error is large due to the wide range of insurance charges (from ~1,000 to over 60,000).

Root Mean Squared Error (RMSE): 5,956.34

The typical prediction error is about $5,956, which is significant but expected given the variability in medical costs.

Interpretation
Model Fit:

With an R² of ~0.81, the model captures most of the relationship between predictors (age, sex, BMI, children, smoker status, region) and insurance charges.

However, ~19% of the variance remains unexplained, suggesting other factors (e.g., medical history, lifestyle, genetic predispositions) may influence charges.

Error Analysis:

The MAE and RMSE values show that while predictions are generally close, there can be large deviations for certain individuals.

This is especially true for smokers and people with very high BMI, where charges can spike dramatically.

Practical Implications:

The model is useful for general predictions and trend analysis (e.g., understanding how smoking or BMI impacts costs).

It is less reliable for precise individual predictions, especially in extreme cases.

Strengths
Captures major cost drivers (smoking, BMI, age).

Provides interpretable coefficients (linear regression is transparent).

Good overall fit with R² > 0.8.

Limitations
Errors remain high in dollar terms (RMSE ≈ $6,000).

Linear regression assumes linear relationships, which may oversimplify complex health cost dynamics.

Outliers (extremely high charges) reduce accuracy.

Recommendations
Feature Engineering: Add interaction terms (e.g., smoker × BMI) to better capture combined effects.

Regularization Models: Try Ridge/Lasso regression to reduce overfitting and handle multicollinearity.

Non-linear Models: Consider tree-based models (Random Forest, Gradient Boosting) for potentially higher accuracy.

Data Enrichment: Include more health-related variables if available (e.g., chronic conditions, exercise habits).

✅ Overall Observation:  
The Linear Regression model performs well, explaining over 80% of the variance in insurance charges. It highlights key cost drivers but still leaves room for improvement, especially in handling extreme cases. While suitable for broad insights, more advanced models may be needed for precise individual-level predictions.